<a href="https://colab.research.google.com/github/onssaadallah/Taxi-Demand-Prediction/blob/main/Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install PyPDF2
! apt-get install poppler-utils 
! pip install pdf2image
! pip install pytesseract
! sudo apt-get install tesseract-ocr-fra

Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.12).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr-fra is already the newest version (4.00~git24-0e00fe6-1.2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
import os
import cv2
from PyPDF2 import PdfFileWriter, PdfFileReader
from pdf2image import convert_from_path
from pytesseract import image_to_string
import json
import warnings
import shutil
import numpy as np

In [ ]:
"""
This class presents the metadata of data extraction.
Metadata is a new storage of specific data extraction from each Bill.
"""

class BillEntity(object): 
   
    def __init__(self): 
        #Num _page
        self.num_page= None 
        #code bill
        self.cod_bill = None
        #Bill title
        self.title = None 
        #MM_YYYY
        self.mm_yyyy = None
        #doc_date
        self.doc_date= None 
        #Full name of the client
        self.cli_name= None
        # Address of client
        self.cli_adr= None
       #total amount
        self.total_amount = None
      
        self.currency_bill='CFA'

    


    # Getter&Setter num_page
    def set_num_page(self,num_page):
        self.num_page == num_page
        return num_page


    def get_num_page(self):
        return self.num_page
    
    numpage= property (get_num_page,set_num_page)

    # Getter&Setter cod_bill
    def set_cod_bill(self,cod_bill):
        self.cod_bill=cod_bill
        return cod_bill


    def get_cod_bill(self):
        return self.cod_bill
    
    codbill = property (get_cod_bill,set_cod_bill)




    # Getter&Setter title   
    def set_title(self,title):
        self.title=title
        return title
        
       
    def get_title(self):
        return self.title

    titlebill = property (get_title,set_title)  


   # Getter&Setter mm_yyyy 
    def set_mm_yyyy (self,mm_yyyy):
        self.mm_yyyy =mm_yyyy 
        return mm_yyyy 
        
        
    def get_mm_yyyy (self):
        return self.mm_yyyy 
    
    mmyyyy = property (get_mm_yyyy ,set_mm_yyyy)  


    # Getter&Setter cli_name
    def set_cli_name(self,cli_name):
        self.cli_name = cli_name
        return cli_name
        
        
    def get_cli_name(self):
        return self.cli_name
    
    cliname = property (get_cli_name,set_cli_name)


     # Getter&Setter cli_adr
    def set_cli_adr(self,cli_adr):
        self.cli_adr = cli_adr
        return cli_adr

   
    def  get_cli_adr (self) :
        return self.cli_adr

    cliadr = property (get_cli_adr,set_cli_adr)  


    # Getter&Setter doc_date
    def set_doc_date(self,doc_date):
        self.doc_date= doc_date
        return doc_date


    def  get_doc_date(self) :
        return self.doc_date 

    docdate = property (get_doc_date,set_doc_date)

    # Getter&Setter total_amount
    def set_total_amount(self,total_amount):
        self.total_amount = total_amount
        return total_amount


    def  get_total_amount (self) :
        return self.atotal_amount    

    totalamount = property (get_total_amount,set_total_amount)


      
    #print the extract data
    def print_pretty(self):   
        print(f'PAGE: {self.num_page}\n')    
        print(f'CODE_BILL: {self.cod_bill}\n')
        print(f'Title: {self.title }\n')
        print(f'MM_YYYY: {self.mm_yyyy}\n')
        print(f'DOC_DATE: {self.doc_date}\n')
        print(f'CLIENT NAME: {self.cli_name}\n')
        print(f'Client Address: {self.cli_adr}\n')
        print(f'total amount: {self.total_amount}\n')
        print(f'currency: {self.currency_bill}\n')
      

In [ ]:
"""
Class UtilsPDF is a util class that splits pdf into single pages and covert them to images.
"""
class UtilsPDF:

    """
    #convert_pdf_to_images
    #Input:
       pdf_filename:the path of pdf file
       list_pages_index:index of pdf file pages
       temp_folder: save all pdf_filename and img_filname in temporary folder
    #Output:
       list_img: return a list of new_img_filename list
    """
    @staticmethod
    def convert_pdf_to_images(pdf_filename,list_page_index,temp_folder,dpi=300,new_img_filename='page%s.jpg'):
        list_img=[]
        images = convert_from_path(pdf_filename,dpi=dpi)
        for i in list_page_index:
            images[i].save(temp_folder+new_img_filename %i, 'JPEG')
            img_page_path=temp_folder+new_img_filename %i
            list_img.append(img_page_path)
        return list_img
          


In [ ]:
class model_MFC:

    PAGES_INDEX_LIST=list(range(98))
    LANG= "fra"
    TEMP_FOLDER='/content/temp_MFC/'
    ROI_STATIC=[('cod_bill',3,[1500,240,500,150]),
                ('title',3,[900,100,700,100]),
               ('mm_yyyy',3,[2000,240,500,150]),
               ('doc_date',3,[900,330,700,100]),                 
                ('cli_name',3,[900,400,700,90]),
                ('cli_adr',3,[900,455,1000,200]),
                ('Total amount',3,[2000,2220,500,150])]


    def __init__(self,filename):    
        self.filename = filename
        self.bill_model = BillEntity()  
        self.roi_static =self.ROI_STATIC
       
 
   # create a local temp folder if not exists
    def setup(self):
        if  os.path.isdir(self.TEMP_FOLDER) == False:
            os.mkdir(self.TEMP_FOLDER)
        pass   
        
    def static_roi(self,page): 
        list_roi_stat= self.roi_static   
        list_info_static=list ()
        for j in range(len(list_roi_stat)): 
            image = cv2.imread(page)  
            #create copy of each image          
            original = np.copy(image)  
            #select the regions of interest  in each image to select a region of  the required data.
            x = list_roi_stat[j][2][0]
            y =list_roi_stat[j][2][1]
            w =list_roi_stat[j][2][2]
            h = list_roi_stat[j][2][3]
            crop_image = original[y:y+h,x:x+w]  
            #extract text from each cropped image using Tesseract (OCR)                              
            text= image_to_string(crop_image,self.LANG)
            list_strings=text.split('\n') 
            #create tuplet of (ID_metadata,list of cleaned data)                   
            list_info_static.append((list_roi_stat[j][0],list_strings))
        return  list_info_static     


    def extract_data(self,page):
        #get all the specific data
        list_data = self.static_roi(page)
        self.bill_model.num_page = page.split('/')[3].split('.')[0]
        try:
            #fill the BillEntity by each id of data extracted
            for j in range(len(list_data)): 
                if list_data[j][0] == 'cod_bill':
                    self.bill_model.codbill = list_data[j][1][0]
                if list_data[j][0] == 'title':
                    self.bill_model.titlebill = list_data[j][1][0]    
                if list_data[j][0] == 'mm_yyyy' :
                    self.bill_model.mm_yyyy= list_data[j][1][0]
                if list_data[j][0] == 'doc_date' :
                    self.bill_model.docdate= list_data[j][1][0]    
                if list_data[j][0] == 'cli_name' :   
                    self.bill_model.cliname= list_data[j][1][0]
                if list_data[j][0] == 'cli_adr' :   
                    self.bill_model.cliadr =list_data[j][1][0] + list_data[j][1][2]
                if list_data[j][0] == 'Total amount' :   
                    self.bill_model.totalamount = list_data[j][1][0] 
                     
            dic=self.bill_model.__dict__     
        
            
        except:
            pass 
        self.bill_model.print_pretty()

   
    #save the extracted data in json file
    def save_doc_to(self,filename):
        with open(filename+'.json', 'w') as json_file:
            json.dump(self.bill_model.__dict__, json_file)
        
    
    # extract specific data from many pages
    def multi_page_extraction(self):
        # create the temporary folder
        self.setup()
        list_page_index=self.PAGES_INDEX_LIST
        pdf_filename=self.filename
        temp_folder=self.TEMP_FOLDER
        #convert the necessary pages of the pdf file to a list of images.
        list_img= UtilsPDF.convert_pdf_to_images(pdf_filename,list_page_index,temp_folder)   
        i=0            
        for page in  list_img:
            self.extract_data(page)
            self.save_doc_to('MFC_'+str (i))
            i=i+1
        self.delete_temp_folder()     
        
       

    #delete of temporary file
    def delete_temp_folder(self):
        shutil.rmtree(self.TEMP_FOLDER) 


In [ ]:
if __name__ == "__main__":
    # Path of the PDF file
    filename='/content/drive/MyDrive/chanllege/facture 12 2020 9715.pdf'
    model=model_MFC(filename)
    model.multi_page_extraction()
  


PAGE: page0

CODE_BILL: 113

Title: YOPOUGON

MM_YYYY: 12/2020

DOC_DATE: 11/12/2020

CLIENT NAME: ORANGE COTE D'IVOIRE

Client Address: PASS CONS NOR RE PRES GE GORE RS CESSE Al PRbd PRINCIPAL rue YAOSSEHIQT YAOSSEHI YOPOUGON

total amount: 126 285

currency: CFA

PAGE: page1

CODE_BILL: 116

Title: YOPOUGON

MM_YYYY: 12/2020

DOC_DATE: 11/12/2020

CLIENT NAME: ORANGE COTE D'IVOIRE

Client Address: PASS CS OS RE RS 6 ee ARE PA PRESSbd BAE ave BANIBAH rue DE LA MOSQUEE Ilot 8 Lot 69LYCEE

total amount: E: CS XS

currency: CFA

PAGE: page2

CODE_BILL: 109

Title: ABIDJAN NORD

MM_YYYY: 12/2020

DOC_DATE: 11/12/2020

CLIENT NAME: ORANGE CI

Client Address: PARUS CONS OS SRave 71 54 54 72 Ilot 24 Lot 2152 PLATEAUX COCODY

total amount: 1 198 300

currency: CFA

PAGE: page3

CODE_BILL: 139

Title: ABIDJAN NORD

MM_YYYY: 12/2020

DOC_DATE: 05/12/2020

CLIENT NAME: ORANGE COTE D'IVOIRE

Client Address: PASS CONS NOR RE PRES GE GORE RS CESSE Al PRave 07 80 70 00 rue 08 27 12 25VALLON 2PLATEAU